<a href="https://colab.research.google.com/github/littleactuary/EAA/blob/main/Markdown_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numbers
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
pd.options.plotting.backend = "plotly"

## **Functions**

In [2]:
# chercher par nom (name) dans une liste d'objets 
# exemple : chercher la loi de rachat d'un model point parmi la liste de toutes les lois de rachats
def search(name, mylist):
  try : 
    return [x for x in mylist if x['name'] == name][0]
  except:
    return 0

# interpolation for converting annual data to monthly data
# input has to be a annual dataFrame (with one column ?)
# type: linear/polynomial/etc with specific order
# non utilisé pour l'instant
def monthly_interp(input, type='linear', order=2) :
  last_row = len(input)
  res = input.set_index(np.arange(1, last_row+1, dtype=int)*12).reindex(np.arange(0, last_row*12+1, dtype=int))
  res.iloc[0,0]=0
  res = res.interpolate(type, order=order).iloc[1:, ]
  return res

# ecart glissant absolu
# ts/tc : taux servi/taux conc de la dernière année, ts_ant/tc_ant : taux servi/taux conc de l'avant-dernière année
# à enrichir : donner la possiblité de paraméter la fenêtre de lissage
def EGA(ts, ts_ant, tc, tc_ant):
  return (ts+ts_ant-tc-tc_ant)/2

def perf(x, basis_x, basis_perf):
  """
  x : series of variable
  basis_x: 1 if 'annual' , 12 if 'monthly'
  basis_perf : 1 if 'annual' , 12 if 'monthly'
  """
  return (x / x.shift(basis_x))**(1/basis_perf)-1
 
def find_eco(data_eco, ref):
  return data_eco[ref]

def eval_var_eco (var, data_eco) :
  res = find_eco(data_eco, var['ref']) 
  window = int(var['window'])*12
  if var['op'] == 'decale':
    return res.shift(window, fill_value = 0) # may be changed ???
  else:
    # must be min, max, moyenne
    res = res.rolling(window)
    if var['op'] == 'moyenne':
      return res.mean()
    elif var['op'] == 'max':
      return res.max()
    else:
      return res.min()

def compute(formula, vars):
  for i in np.arange(0, len(vars)):
    formula = formula.replace('x{}'.format(i+1), 'vars[{}]'.format(i))
  return eval(formula)

## **Taux concurrentiel (SI) et OTS (politique de résultat)**




# Taux de référence 

* (Niveau 0) var : {
  'op': choix dans ['decale', 'min', 'max', 'moyenne'] 
  'window': Number , 
  'ref': une variable économique , 
  'desc': 'decale_0_taux_long'
  }
*  (Niveau 1) Taux_ref : formule à partir des vars --> on peut définir le taux concurrentiel avec
*  (Niveau 2) Rate_func : a * Taux_ref + b --> pas vraiment nécessaire. Mais ça facilite la création des OTS par model point à partir d'un seul Taux_ref (b est généralement le chargement sur encours)
* (Niveau 3) OTS : objectif, planchers, plafonds

In [3]:
# taux de référence : name, vars [x1, x2, x3], formula
# eval(var_bdd) avec var_bdd tableau des valeurs x1, x2, x3 projetées dans le temps
class Taux_ref:
  def __init__(self, name, vars, formula):
    self.name = name
    self.vars = vars
    self.formula = formula


  def eval(self, var_bdd):
    xNames = list(map(lambda x: x['desc'] , self.vars))
    vars = var_bdd[xNames]
    duree = var_bdd.shape[0]
    res = np.zeros(duree)
    for i in np.arange(0, duree):
      res[i] = compute(self.formula, vars.iloc[i, :])
    vars.insert(0, "output", res, True)
    return vars
    

# coef A + taux de référence + coefB
class Taux_fct :
    def __init__(self, a, ref, b):
        self.a = a
        self.b = b
        self.ref = ref  
    def __str__(self):
      return f"{self.a :.2%} * {self.ref.name} {self.b:+.2%}"
    
    def print(self):
      return f"{self}"

    def eval(self, var_bdd):
      return self.a * self.ref.eval(var_bdd)['output'] + self.b



# objectif de taux servis
# target : taux cible Taux_fct
# floor1 : taux plancher 1 -> Taux_fct
# floor2 : taux plancher 2 -> propre au produit
# cap1 : taux plafond 1 -> Taux_fct
# cap2 : taux plancher 2 -> propre au produit
class OTS:
    def __init__(self, target, floor1, floor2, cap1, cap2):
        self.target = target
        self.floor1 = floor1
        self.floor2 = floor2
        self.cap1 = cap1
        self.cap2 = cap2
    
    def print(self):
      return {
         'objectif': self.target.print(),
         'plancher 1': self.floor1.print(),
         'plancher 2': self.floor2.print(),
         'plafond 1': self.cap1.print(),
         'plafond 2': self.cap2.print()
      } 

    def eval(self, var_bdd):
      target = self.target.eval(var_bdd)
      floor1 = self.floor1.eval(var_bdd)
      floor2 = self.floor2.eval(var_bdd)
      cap1 = self.cap1.eval(var_bdd)
      cap2 = self.cap2.eval(var_bdd)
      res = pd.DataFrame({'target': target, 'floor1': floor1, 'floor2': floor2, 'cap1': cap1, 'cap2': cap2})
      floor = res[['floor1', 'floor2']].max(axis=1)
      cap = res[['cap1', 'cap2']].min(axis=1)
      res.insert(1, 'floor', floor, True)
      res.insert(4, 'cap', cap, True)
      res=res.drop(['floor1', 'floor2', 'cap1', 'cap2'], axis=1)
      res['cap'] = res[['floor','cap']].max(axis=1)
      return res




## <strong>Classe</strong>

Classe Epargne euro

In [6]:
class Epargne_euro:
  def __init__(self, name, stock, TMGA, PB_contractuelle, charg_comm, table_morta, loi_echeance, loi_rachat_struc, loi_rachat_conj, taux_conc):
    self.name = name 
    self.stock = stock # une ligne par année de souscription (on en déduit date début et fin de souscription). colonne 1 : PM, colonne 2 : versement potentiel, colonne 3 : TMG moyen
    self.tmga = TMGA
    self.csg_opt = csg_opt # CSG ? (true/false)
    self.pb_min_regl_option # inclus dans calcul PB min regl ? (true/false)
    self.pb_contractuelle = PB_contractuelle # {'pdt_fi': 0, 'res_tech_pos': 0, 'res_tech_neg':0}
    self.charg_comm = charg_comm # {'chgt_primes':0, 'comm_primes':0, 'chgt_encours':0, 'comm_encours':0, 'pen_rachat': 0 }
    self.table_morta = table_morta # année de simu en ligne, année de souscription en colonne
    self.loi_echeance = loi_echeance # comme rachat (ou décès, à voir...)
    self.loi_rachat_struc = search(loi_rachat_struc, liste_rachats_struc) # nom de la loi
    self.loi_rachat_conj = search(loi_rachat_conj, liste_rachats_conj) # nom de la loi. mapping à faire avec la liste des lois de rachats conjoncturels
    self.taux_conc = search(taux_conc, liste_taux_ref) # nom du taux de référence. Mapping à faire avec la liste des taux de référence
    
    #self.OTS = None

  def set_OTS(self):
    # OTS défini dans la politique de résultats
    self.OTS = OTS
    



In [ ]:
#test


rachats_structurels = pd.read_excel('rachats_structurels.xlsx', header=0, index_col=0)
echeances = pd.read_excel('echeances.xlsx', header=0, index_col=0)
forcage_TS = pd.read_excel('forcage_TS.xlsx', header=0, index_col=0)
frais_generaux = pd.read_excel('frais_generaux.xlsx', header=0, index_col=0)
fiscalite = pd.read_excel('fiscalite.xlsx', header=0, index_col=0)
rachats_structurels = pd.read_excel('rachats_structurels.xlsx', header=0, index_col=0)

name = "Winalto"
stock = pd.read_excel('stock.xlsx', header=0, index_col=0)
TMGA = 0.01
PB_contractuelle = {'pdt_fi': 0.85, 'res_tech_pos': 0.9, 'res_tech_neg':1}
charg_comm = pd.read_excel('comm_chgt.xlsx', header=0, index_col=0)
table_morta =  pd.read_excel('table_morta.xlsx', header=0, index_col=0)
loi_echeance = pd.read_excel('echeances.xlsx', header=0, index_col=0)
loi_rachat_struc = pd.read_excel('rachats_structurels.xlsx', header=0, index_col=0)
loi_rachat_conj = 
mp1 = Epargne_euro(name, stock, TMGA, PB_contractuelle, charg_comm, table_morta, loi_echeance, loi_rachat_struc, loi_rachat_conj, taux_conc)